In [1]:
pip install torchvision

     |████████████████████████████████| 17.4 MB 4.1 MB/s eta 0:00:01     |██████████████████▋             | 10.1 MB 4.1 MB/s eta 0:00:02
     |████████████████████████████████| 804.1 MB 39.8 MB/s eta 0:00:01�████████▎                   | 308.2 MB 3.0 MB/s eta 0:02:48�██████████████▏               | 406.0 MB 3.0 MB/s eta 0:02:11██████▌              | 439.3 MB 2.7 MB/s eta 0:02:17 |██████████████████▍             | 463.4 MB 2.7 MB/s eta 0:02:08�█████████████████▋             | 468.5 MB 2.7 MB/s eta 0:02:06 519.3 MB 2.5 MB/s eta 0:01:55█████████████████▏          | 531.9 MB 2.5 MB/s eta 0:01:49     |███████████████████████         | 580.1 MB 73.6 MB/s eta 0:00:04�███████████        | 605.1 MB 89.5 MB/s eta 0:00:038.8 MB 89.5 MB/s eta 0:00:03██████████████████████▉       | 622.7 MB 89.5 MB/s eta 0:00:03███████       | 627.5 MB 89.5 MB/s eta 0:00:02�██████████████████▍      | 637.2 MB 52.2 MB/s eta 0:00:04�██▊      | 645.9 MB 52.2 MB/s eta 0:00:04MB/s eta 0:00:03��██▋     | 667.6 MB 52.2 MB

In [2]:
pip install markdown

     |████████████████████████████████| 97 kB 3.1 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

     |████████████████████████████████| 394.3 MB 27 kB/s s eta 0:00:01
     |████████████████████████████████| 462 kB 49.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 69.6 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.5 MB/s  eta 0:00:01
     |████████████████████████████████| 10.6 MB 73.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 68.2 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 4.8 MB/s  eta 0:00:01
     |████████████████████████████████| 129 kB 54.7 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 6.3 MB/s  eta 0:00:01
     |████████████████████████████████| 2.9 MB 55.2 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 55.8 MB/s eta 0:00:01
     |████████████████████████████████| 142 kB 46.3 MB/s eta 0:00:01
     |████████████████████████████████| 298 kB 50.3 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 63.9 MB/s eta 0:00:01
     |████████████████████████████

In [4]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [5]:
# pip install --user --upgrade tensorflow-gpu

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
import tensorflow as tf
import random
import numpy as np
from tensorflow.keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from tensorflow.keras import utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
def getZerosAndOnes(fileName, electrod, trainslabel):
    data = pd.read_csv(fileName)
    session= data[[electrod,'FeedBackEvent']]
    labels = pd.read_csv(trainslabel) 
    eventsIndexes=[]
    for index, row in session.iterrows():
        if row['FeedBackEvent']==1:
            eventsIndexes.append(index)
    eventsValues=[]
    for index, row in labels.iterrows():
        if row['IdFeedBack'].startswith('S02_Sess01'):
            eventsValues.append(row['Prediction'])   
    dataEpoches=[]
    for (index, val) in enumerate(eventsIndexes):
        if eventsValues[index]==1:
            dataEpoches.append(session[val-120:val+80])
    moreDataEpoches=[]
    for (index, val) in enumerate(eventsIndexes):
        if index>0 and index<len(eventsIndexes)-1:
            count=1
            while val+80+count*200<eventsIndexes[index+1]-120:
                count+=1
                moreDataEpoches.append(session[val+80+count*200:val+80+count*200+200])
    Xm=[]
    for row in moreDataEpoches:
        Xm.append(row[electrod])
    Xm=np.array(Xm)
    Xs=[]
    for row in dataEpoches:
        Xs.append(row[electrod])
    Xs=np.array(Xs)
    y_dataZero=np.zeros(len(Xm))
    y_dataOne=np.ones(len(Xs))
    x_dataZero=Xm
    x_dataOne=Xs
    return x_dataOne, y_dataOne, x_dataZero, y_dataZero

In [4]:
x_dataOne, y_dataOne, x_dataZero, y_dataZero=getZerosAndOnes("data.csv", 'Fp2', "TrainLabels.csv")


In [5]:
print(x_dataOne.shape)
print(y_dataOne.shape)
print(x_dataZero.shape)
print(y_dataZero.shape)

(50, 200)
(50,)
(493, 200)
(493,)


In [111]:
def load_data_from_arrays(one, zero, y_one, y_zero, train_test_split=0.9):
    data_size_one = len(one)
    data_size_zero = len(zero)
    data_size=data_size_one+data_size_zero
    test_size = int(data_size - round(data_size_zero * train_test_split)- round(data_size_one * train_test_split))
    print("Data size: {}".format(data_size))
    print("\nTest size: {}".format(test_size))
    
    data_one=one[0:round(data_size_one * train_test_split)]
    data_zero=zero[0:round(data_size_zero * train_test_split)]
    data_train=np.concatenate((data_one, data_zero), axis=0)
    train=pd.DataFrame(data_train)
    
    print(data_one.shape)
    print(data_zero.shape)
    print(train.shape)
    print("\nTraining set:")
    x_train = train
    
    data_one=y_one[0:round(data_size_one * train_test_split)]
    data_zero=y_zero[0:round(data_size_zero * train_test_split)]
    data_train=np.concatenate((data_one, data_zero), axis=0)
    train=pd.DataFrame(data_train)
    
    print(data_one.shape)
    print(data_zero.shape)
    print(train.shape)
    print("\t - x_train: {}".format(len(x_train)))
    y_train = train
    print("\t - y_train: {}".format(len(y_train)))
    
    
    data_one=one[round(data_size_one * train_test_split):data_size_one]
    data_zero=zero[round(data_size_zero * train_test_split):data_size_zero]
    data_test=np.concatenate((data_one, data_zero), axis=0)
#     random.shuffle(data_test)
    test=pd.DataFrame(data_test)
    
    print(data_one.shape)
    print(data_zero.shape)
    print(train.shape)
    print("\nTesting set:")
    x_test = test
    
    data_one=y_one[round(data_size_one * train_test_split):data_size_one]
    data_zero=y_zero[round(data_size_zero * train_test_split):data_size_zero]
    data_test=np.concatenate((data_one, data_zero), axis=0)
#     random.shuffle(data_test)
    test=pd.DataFrame(data_test)
    
    print(data_one.shape)
    print(data_zero.shape)
    print(train.shape)
    print("\t - x_test: {}".format(len(x_test)))
    y_test = test
    print("\t - y_test: {}".format(len(y_test)))
    return x_train, y_train, x_test, y_test

In [112]:
x_train, y_train, x_test, y_test=load_data_from_arrays(x_dataOne, x_dataZero,y_dataOne,y_dataZero,0.75)

Data size: 543

Test size: 135
(38, 200)
(370, 200)
(408, 200)

Training set:
(38,)
(370,)
(408, 1)
	 - x_train: 408
	 - y_train: 408
(12, 200)
(123, 200)
(408, 1)

Testing set:
(12,)
(123,)
(408, 1)
	 - x_test: 135
	 - y_test: 135


In [113]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(408, 200)
(408, 1)
(135, 200)
(135, 1)


In [114]:
x_test[0:20]

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-101.733265,-139.240568,-97.917767,-139.829877,-100.322393,-138.770218,-115.542448,-160.454885,-100.114875,-134.483297,...,-87.619789,-143.603179,-80.939364,-120.716926,-84.040291,-140.144070,-96.928863,-134.754201,-81.298266,-124.914111
1,-84.322207,-127.746642,-71.094311,-98.359902,-83.377766,-92.160924,-58.027053,-96.045966,-77.958191,-112.389423,...,-82.962464,-145.628450,-97.354226,-150.343761,-97.435157,-120.500817,-100.963845,-130.664413,-141.422377,-117.821004
2,-166.529551,-206.608882,-153.474955,-187.506455,-184.270211,-222.805770,-174.537467,-201.116244,-164.702151,-229.477561,...,-125.653047,-178.543096,-123.974455,-159.680132,-117.823506,-171.212184,-126.446699,-161.664737,-118.996358,-141.024907
3,-70.968223,-112.679917,-60.152138,-94.373704,-77.187614,-104.983959,-60.714545,-103.603202,-58.626328,-116.295768,...,-102.139372,-143.679908,-94.543457,-130.485688,-70.471691,-130.946968,-92.917713,-114.906294,-82.097692,-123.808495
4,-86.216641,-129.539186,-112.967663,-124.443308,-94.815861,-140.262840,-84.705954,-146.929518,-88.585448,-151.895873,...,-94.696825,-131.888015,-109.964807,-121.841049,-94.291710,-129.887075,-81.354694,-159.812899,-97.985187,-112.505042
5,-73.463330,-86.257088,-72.933652,-82.727650,-88.501427,-102.542004,-104.068591,-81.203286,-67.391801,-83.273792,...,-96.520207,-51.124243,-68.107658,-68.917702,-73.078122,-54.386154,-42.223906,-57.538756,-58.585633,-68.286980
6,-74.906031,-35.103065,-52.952942,-45.142145,-60.180167,-25.315510,-80.059185,-44.516315,-70.698497,-32.534102,...,-50.939546,-1.678183,-52.064274,-8.852121,-54.853505,-42.647193,-70.378765,-19.094349,-57.182562,-9.778215
7,-47.796658,-12.215361,-36.277694,-7.169779,-15.010222,-22.377045,-32.328791,-33.992927,-41.115776,-3.694970,...,-22.834153,16.084900,17.518301,4.647985,-12.318238,4.188059,-6.298668,20.301567,-2.082662,-0.223428
8,-21.572384,-21.907622,-23.063556,0.949898,-13.398830,-29.188888,-14.041197,-40.407085,-17.794733,-24.665539,...,-8.346886,-15.174896,-15.510243,-19.692785,-6.706304,-15.616092,-16.440146,-24.024663,-7.242324,-40.266411
9,-31.031568,7.296150,1.926114,8.400427,-41.251506,-20.700665,-53.998712,-2.128057,-29.753433,-1.259563,...,-21.581169,-11.171668,-10.752915,34.029068,5.708468,34.079929,6.800649,27.407586,17.199127,13.225218


In [115]:
y_test[0:20]

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0
6,1.0
7,1.0
8,1.0
9,1.0


In [116]:
from sklearn.neighbors import KNeighborsClassifier

In [117]:
x_train_d=[]
for val in x_train.T:
    buf=[]
    i=0
#     print(x_train.T[val])
    for vall in x_train.T[val]:
        if i%8==0:
            buf.append(vall)
        i+=1
    x_train_d.append(buf)
x_train_d=pd.DataFrame(x_train_d)
print(x_train_d.shape)

(408, 25)


In [118]:
x_test_d=[]
for val in x_test.T:
    buf=[]
    i=0
#     print(x_train.T[val])
    for vall in x_test.T[val]:
        if i%8==0:
            buf.append(vall)
        i+=1
    x_test_d.append(buf)
x_test_d=pd.DataFrame(x_test_d)
print(x_test_d.shape)

(135, 25)


In [119]:
x_train_d

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,990.932702,976.333103,1019.090094,981.513148,974.498632,991.480573,980.500528,989.435094,968.609672,981.945547,...,979.638619,990.937619,977.953398,984.161158,988.210557,976.274092,986.466962,979.049888,993.676019,981.070808
1,905.919767,916.545341,921.541297,902.423787,896.067984,914.725388,888.880353,903.973058,906.316006,890.893093,...,872.008242,868.903258,869.987481,883.449871,884.502596,883.728086,864.382887,849.687361,858.660594,865.603304
2,529.511870,505.509949,511.395967,522.034628,515.030236,527.207105,528.559674,530.757924,529.588085,534.569561,...,504.661776,487.698645,478.464324,506.364043,498.252942,508.750588,500.964617,492.492743,489.903622,451.430929
3,333.706603,322.849566,326.184883,293.363579,321.239072,310.133451,321.800749,322.926714,304.211506,284.907583,...,275.487436,279.364158,264.113322,269.292955,267.483932,254.039053,259.748245,275.479024,292.476954,275.771744
4,183.991489,181.889978,190.535397,186.211995,203.107304,187.243255,182.347609,183.411922,181.729228,188.360838,...,186.608261,169.573652,220.982289,212.244465,214.403335,206.660784,214.738990,222.953363,215.431766,212.055431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,-186.868018,-186.457178,-188.798013,-186.856528,-197.439219,-180.884324,-198.755923,-184.271460,-201.030433,-197.398653,...,-172.196594,-194.997757,-208.489986,-192.987189,-186.269355,-181.721761,-192.915965,-201.237704,-164.974276,-187.918195
404,-170.754725,-181.334010,-207.052660,-174.961888,-210.095019,-202.496766,-188.979921,-192.642140,-198.103674,-195.693801,...,-205.715709,-205.159052,-194.820938,-192.055992,-195.706508,-200.556523,-211.320990,-194.683587,-214.330803,-181.948627
405,-180.591908,-177.677296,-194.475921,-179.726375,-196.793327,-202.890512,-189.596171,-180.799941,-194.918704,-195.020127,...,-175.155629,-174.790238,-199.767580,-198.633011,-164.847210,-189.401898,-180.449428,-208.255376,-188.892311,-187.891199
406,-177.764519,-174.442911,-179.945368,-178.723233,-179.481137,-191.835964,-194.040782,-156.364452,-181.382348,-183.290530,...,-183.123981,-166.492030,-162.508799,-174.676342,-164.930655,-186.408415,-177.273011,-196.142462,-175.447717,-197.253625


In [120]:
x_test_d

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-101.733265,-100.114875,-103.455229,-90.897774,-102.768467,-105.789906,-97.628993,-78.588352,-107.869976,-98.616899,...,-98.610497,-116.083343,-103.328914,-117.234372,-94.312884,-102.975944,-91.652374,-80.797755,-77.934646,-80.939364
1,-84.322207,-77.958191,-115.343852,-104.596451,-98.911159,-105.128366,-89.885591,-103.945133,-103.772006,-102.075076,...,-114.747496,-109.005208,-117.401174,-123.837197,-87.983398,-93.574490,-101.261472,-114.314769,-105.728723,-97.354226
2,-166.529551,-164.702151,-166.340459,-170.690138,-155.741560,-166.925877,-164.536860,-161.136240,-162.641908,-151.665161,...,-149.656620,-152.054526,-175.868144,-143.390361,-125.174607,-146.854442,-133.495186,-86.888933,-112.977037,-123.974455
3,-70.968223,-58.626328,-93.371820,-86.314751,-101.626130,-82.550788,-84.850121,-113.999099,-93.014830,-88.740953,...,-106.106026,-101.636951,-104.825242,-95.062018,-93.834166,-102.290281,-74.199551,-86.957341,-105.547333,-94.543457
4,-86.216641,-88.585448,-91.542803,-108.854188,-100.535285,-83.474177,-99.574957,-113.113213,-88.183637,-98.262446,...,-103.730663,-96.605807,-105.917634,-111.120518,-85.288190,-104.595653,-100.374847,-115.938823,-104.035077,-109.964807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,-29.485479,-25.536856,-14.857120,-43.688060,-27.189171,-52.215456,-21.828230,-40.650415,-24.476253,-35.278364,...,-32.275676,-22.594650,-39.379661,-44.536271,-31.869525,-27.814473,-26.061991,-29.366205,-24.184416,-22.220003
131,-53.127187,-22.153850,-31.381700,-25.479431,-25.615062,-21.823017,-19.959061,-4.724306,-34.446821,-33.581191,...,-16.278486,-15.550050,-39.087877,-45.276807,-38.045849,-25.872235,-39.024483,-0.963358,-3.707955,-14.720359
132,-31.455983,-28.413026,-21.084999,7.389879,-9.885834,-28.840687,-22.986926,-2.790782,-33.413664,-35.872830,...,4.385282,-11.207430,-8.277214,-8.032250,-14.458386,-10.374806,-5.580013,31.152459,13.599056,-2.833614
133,-7.663974,1.499224,12.440612,-5.315280,-1.152517,-16.287494,-12.900398,-17.173202,-11.238986,-13.218667,...,8.896659,-17.659984,-36.924721,-19.408552,-26.314406,-17.863421,-25.295472,-12.903940,-6.220219,-22.408031


In [121]:
x_train=x_train_d
x_test=x_test_d
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(408, 25)
(408, 1)
(135, 25)
(135, 1)


In [122]:
x_test[0]

0     -101.733265
1      -84.322207
2     -166.529551
3      -70.968223
4      -86.216641
          ...    
130    -29.485479
131    -53.127187
132    -31.455983
133     -7.663974
134    -24.737375
Name: 0, Length: 135, dtype: float64

In [126]:
inputs = keras.Input(shape=(25,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
y = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(1, activation='sigmoid', name='output')(y)

model = keras.Model(inputs=inputs, outputs=outputs)

In [127]:
# x_train = (np.array(x_train)).reshape(406, 200).astype('float32')
# x_test = (np.array(x_test)).reshape(135, 200).astype('float32')
# y_train = (np.array(y_train)).reshape(406, 1).astype('float32')
# y_test = (np.array(y_test)).reshape(135, 1).astype('float32')
x_test.T[4]

0     -86.216641
1     -88.585448
2     -91.542803
3    -108.854188
4    -100.535285
5     -83.474177
6     -99.574957
7    -113.113213
8     -88.183637
9     -98.262446
10    -90.146025
11    -97.471624
12    -95.369965
13   -100.253305
14    -97.611702
15   -103.730663
16    -96.605807
17   -105.917634
18   -111.120518
19    -85.288190
20   -104.595653
21   -100.374847
22   -115.938823
23   -104.035077
24   -109.964807
Name: 4, dtype: float64

In [132]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['sparse_categorical_accuracy'])

print('# Обучаем модель на тестовых данных')
history = model.fit(x_train_d, y_train,
                    batch_size=64,
                    epochs=30,
                    # Мы передаем валидационные данные для
                    # мониторинга потерь и метрик на этих данных
                    # в конце каждой эпохи
                    validation_data=(x_test_d, y_test))

# Обучаем модель на тестовых данных
Epoch 1/30
7/7 [==============================] - 1s 46ms/step - loss: 16.0888 - sparse_categorical_accuracy: 0.8983 - val_loss: 0.9586 - val_sparse_categorical_accuracy: 0.9111
Epoch 2/30
7/7 [==============================] - 0s 10ms/step - loss: 3.4103 - sparse_categorical_accuracy: 0.9167 - val_loss: 0.8379 - val_sparse_categorical_accuracy: 0.9111
Epoch 3/30
7/7 [==============================] - 0s 10ms/step - loss: 2.3229 - sparse_categorical_accuracy: 0.9130 - val_loss: 0.5993 - val_sparse_categorical_accuracy: 0.9111
Epoch 4/30
7/7 [==============================] - 0s 12ms/step - loss: 2.4382 - sparse_categorical_accuracy: 0.9067 - val_loss: 0.5630 - val_sparse_categorical_accuracy: 0.9111
Epoch 5/30
7/7 [==============================] - 0s 11ms/step - loss: 4.0143 - sparse_categorical_accuracy: 0.9009 - val_loss: 0.4785 - val_sparse_categorical_accuracy: 0.9111
Epoch 6/30
7/7 [==============================] - 0s 14ms/step - loss: 3.0040 

In [133]:
results = model.evaluate(x_test_d, y_test, batch_size=128)
print('test loss, test acc:', results)
predictions = model.predict(x_test_d)
print('размерность прогнозов:', predictions.shape)

2/2 [==============================] - 0s 3ms/step - loss: 0.3122 - sparse_categorical_accuracy: 0.9111
test loss, test acc: [0.3122292757034302, 0.9111111164093018]
размерность прогнозов: (135, 1)


In [130]:
predictions=np.round(predictions,1)

In [54]:
for (index, val) in enumerate(y_test):
    print(val," : ",predictions[index])

0  :  [2.4434259e-17]


In [99]:
for (index, val) in enumerate(y_test[10:]):
    print(val," : ",predictions[index])

0  :  [0.06248954]


In [134]:
predictions[:15]

array([[0.07004273],
       [0.01432568],
       [0.00932354],
       [0.7952069 ],
       [0.29077488],
       [0.40293515],
       [0.01925334],
       [0.05227357],
       [0.34775943],
       [0.3001107 ],
       [0.01858458],
       [0.00180599],
       [0.00094044],
       [0.02372825],
       [0.00631481]], dtype=float32)

In [106]:
y_test

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
130,0.0
131,0.0
132,0.0
133,0.0


In [135]:
predictions

array([[7.00427294e-02],
       [1.43256783e-02],
       [9.32353735e-03],
       [7.95206904e-01],
       [2.90774882e-01],
       [4.02935147e-01],
       [1.92533433e-02],
       [5.22735715e-02],
       [3.47759426e-01],
       [3.00110698e-01],
       [1.85845792e-02],
       [1.80599093e-03],
       [9.40442085e-04],
       [2.37282515e-02],
       [6.31481409e-03],
       [8.63465667e-03],
       [1.51336193e-04],
       [2.57691741e-03],
       [6.44472241e-03],
       [8.48537683e-03],
       [3.78488302e-02],
       [1.00636184e-02],
       [1.89096034e-02],
       [7.33211637e-03],
       [4.91589308e-04],
       [1.50657892e-02],
       [1.63156271e-01],
       [2.10078061e-01],
       [6.77113533e-02],
       [1.79840470e-07],
       [4.93404269e-02],
       [3.75946075e-01],
       [8.84651542e-02],
       [9.39428806e-04],
       [1.52266026e-03],
       [7.69740343e-03],
       [1.49649759e-06],
       [6.02424145e-04],
       [2.03551054e-02],
       [1.61744254e-06],
